In [1]:
import os
import pandas as pd
import re
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
session = "Ses01F_impro03"
f = f"IEMOCAP_full_release/Session1/dialog/transcriptions/{session}.txt"
video = f"IEMOCAP_full_release/Session1/dialog/avi/DivX/{session}.avi"

with open(f, "r") as file:
    lines = file.readlines()

d = {"start": [], "end": [], "speaker": [], "utterance": []}

for i, line in enumerate(lines):
    speaker_info, utterance = line.split(":")[0], line.split(":")[1]
    pattern = r"(F|M)\d+\s\[(\d+\.\d+)-(\d+\.\d+)\]"
    match = re.search(pattern, speaker_info)
    if match is None:
        continue
    start = match.group(2)
    end = match.group(3)
    d["start"].append(start) 
    d["end"].append(end)
    d["speaker"].append(match.group(1))
    d["utterance"].append(utterance.strip())

    clip = VideoFileClip(video).subclip(start, end)  # This form is a Z.X.four.\n'
    clip.write_videofile(f"{session}_{i}.mp4")

df = pd.DataFrame(d)
# df.to_csv(f"IEMOCAP_full_release/Session1/dialog/transcriptions/{session}.csv")

Moviepy - Building video Ses01F_impro03_0.mp4.
MoviePy - Writing audio in Ses01F_impro03_0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Ses01F_impro03_0.mp4



Moviepy - Done !
Moviepy - video ready Ses01F_impro03_0.mp4


Moviepy - Building video Ses01F_impro03_1.mp4.
MoviePy - Writing audio in Ses01F_impro03_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Ses01F_impro03_1.mp4



...

Moviepy - Done !
Moviepy - video ready Ses01F_impro03_51.mp4
Moviepy - Building video Ses01F_impro03_52.mp4.
MoviePy - Writing audio in Ses01F_impro03_52TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Ses01F_impro03_52.mp4



Moviepy - Done !
Moviepy - video ready Ses01F_impro03_52.mp4
Moviepy - Building video Ses01F_impro03_53.mp4.
MoviePy - Writing audio in Ses01F_impro03_53TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Ses01F_impro03_53.mp4



Moviepy - Done !
Moviepy - video ready Ses01F_impro03_53.mp4
